# Gráfico de Passes Interativo

## Importação das Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from statsbombpy import sb
from mplsoccer.pitch import Pitch
import ipywidgets as widgets

## Seleção da Partida

In [2]:
events = sb.events(3773369)
passes = events[events['type'] == 'Pass']

In [3]:
passes.sample()

,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,block_offensive,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,...,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,timestamp,type,under_pressure
170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Barcelona,00:10:05.960,Pass,NaN


In [4]:
passes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1357 entries, 6 to 1362
Data columns (total 89 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ball_receipt_outcome            0 non-null      object 
 1   ball_recovery_recovery_failure  0 non-null      object 
 2   block_deflection                0 non-null      object 
 3   block_offensive                 0 non-null      object 
 4   carry_end_location              0 non-null      object 
 5   clearance_aerial_won            0 non-null      object 
 6   clearance_body_part             0 non-null      object 
 7   clearance_head                  0 non-null      object 
 8   clearance_left_foot             0 non-null      object 
 9   clearance_other                 0 non-null      object 
 10  clearance_right_foot            0 non-null      object 
 11  counterpress                    5 non-null      object 
 12  dribble_nutmeg                  0 

## Preparação dos Dados

In [5]:
x = [x[0] for x in passes['location']]
y = [y[1] for y in passes['location']]
end_x = [x[0] for x in passes['pass_end_location']]
end_y = [y[1] for y in passes['pass_end_location']]
passes = passes.assign(x=x, y=y, end_x=end_x, end_y=end_y)

### Importação da Tabela de xT

In [6]:
xt_df = pd.read_csv('xT_metric.csv', header=None)
xt_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.006383,0.007796,0.008449,0.009777,0.011263,0.012483,0.014736,0.017451,0.021221,0.027563,0.034851,0.037926
1,0.007501,0.008786,0.009424,0.010595,0.012147,0.013845,0.016118,0.018703,0.024015,0.029533,0.040670,0.046477
2,0.008880,0.009777,0.010013,0.011105,0.012692,0.014291,0.016856,0.019351,0.024122,0.028552,0.054911,0.064426
3,0.009411,0.010827,0.010165,0.011324,0.012626,0.014846,0.016895,0.019971,0.023851,0.035113,0.108051,0.257454
4,0.009411,0.010827,0.010165,0.011324,0.012626,0.014846,0.016895,0.019971,0.023851,0.035113,0.108051,0.257454
5,0.008880,0.009777,0.010013,0.011105,0.012692,0.014291,0.016856,0.019351,0.024122,0.028552,0.054911,0.064426
6,0.007501,0.008786,0.009424,0.010595,0.012147,0.013845,0.016118,0.018703,0.024015,0.029533,0.040670,0.046477
7,0.006383,0.007796,0.008449,0.009777,0.011263,0.012483,0.014736,0.017451,0.021221,0.027563,0.034851,0.037926


In [7]:
xT = np.array(xt_df)

In [8]:
xT_rows, xT_cols = xT.shape

In [9]:
passes['x1_bin'] = pd.cut(passes.loc[:, 'x'], bins=xT_cols, labels=False)
passes['y1_bin'] = pd.cut(passes.loc[:, 'y'], bins=xT_rows, labels=False)
passes['x2_bin'] = pd.cut(passes.loc[:, 'end_x'], bins=xT_cols, labels=False)
passes['y2_bin'] = pd.cut(passes.loc[:, 'end_y'], bins=xT_rows, labels=False)

passes['start_zone_value'] = passes[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
passes['end_zone_value'] = passes[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
passes['xT%'] = round((passes.loc[:, 'end_zone_value'] / passes.loc[:, 'start_zone_value'] - 1), 2)

In [10]:
passes.sample()

,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,block_offensive,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,...,y,end_x,end_y,x1_bin,y1_bin,x2_bin,y2_bin,start_zone_value,end_zone_value,xT%
277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.0,30.2,32.7,3,1,3,3,0.010595,0.011324,0.07


In [11]:
l = list(passes['pass_outcome'].unique())
l = l[1:]
l

['Out', 'Incomplete', 'Unknown', 'Pass Offside']

In [12]:
passes = passes[~passes['pass_outcome'].isin(l)]

In [13]:
passes = passes[passes['play_pattern']=='Regular Play']

In [14]:
passes.team.unique()

array(['Barcelona', 'Huesca'], dtype=object)

## Gráfico Interativo

In [15]:
def plot_interact(team, minute):
    passes_final = passes[(passes['team']==team) & (passes['minute']<=minute) & (passes['xT%']>=passes['xT%'].max()*0.1)]
    pitch = Pitch(pitch_type='statsbomb', pitch_color='#22312b', line_color='#c7d5cc', goal_type='box')
    fig, ax = pitch.draw(figsize=(10, 7))
    pitch.scatter(passes_final.x, passes_final.y, c='r', cmap=['Blues'], ax=ax, s=100);
    pitch.arrows(passes_final.x, passes_final.y, passes_final.end_x, passes_final.end_y, color='white', width=2, ax=ax)
    ax.set_title(f'Match of {passes.team.unique()[0]} Vs {passes.team.unique()[1]} Passes up to {minute} Minutes', fontsize=20);

In [16]:
widgets.interact(plot_interact, team=widgets.Dropdown(options=passes['team'].unique()), 
                 minute=widgets.Play(min=0, max=passes['minute'].max()), interval=1000);

interactive(children=(Dropdown(description='team', options=('Barcelona', 'Huesca'), value='Barcelona'), Play(v…